# Sales Forecasting Model

This notebook develops and evaluates time series forecasting models using Facebook Prophet.

## Objectives
- Train Prophet forecasting model
- Evaluate model performance
- Generate future predictions
- Analyze forecast components


In [ ]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from src.forecasting_model import SalesForecaster
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('✓ Libraries imported')


## 1. Load Data


In [ ]:
df = pd.read_csv('../data/raw/ethiopia_sales_raw.csv')
df['date'] = pd.to_datetime(df['date'])

print(f'Loaded {len(df)} transactions')
print(f'Date range: {df["date"].min()} to {df["date"].max()}')
df.head()


## 2. Prepare Data for Forecasting


In [ ]:
# Aggregate daily sales
df_daily = df.groupby('date')['total_sales'].sum().reset_index()
df_daily.columns = ['ds', 'y']

print(f'Daily aggregated data: {len(df_daily)} days')
print(f'\nSample:')
df_daily.head()


In [ ]:
# Visualize time series
plt.figure(figsize=(14, 6))
plt.plot(df_daily['ds'], df_daily['y'], linewidth=1.5, alpha=0.7)
plt.title('Daily Sales Time Series', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Total Sales (ETB)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 3. Train Prophet Model


In [ ]:
# Initialize and train using our custom forecaster
forecaster = SalesForecaster()
forecaster.prepare_data(df, test_size=90)
forecaster.train_model()

print('✓ Model trained successfully!')


## 4. Evaluate Model Performance


In [ ]:
# Evaluate model
metrics = forecaster.evaluate_model()

print('\nModel Performance Summary:')
for metric, value in metrics.items():
    print(f'{metric}: {value:,.2f}')


## 5. Generate Forecast


In [ ]:
# Generate 90-day forecast
forecast = forecaster.forecast(periods=90)

print(f'✓ Forecast generated for 90 days')
print(f'\nForecast shape: {forecast.shape}')
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


## 6. Visualize Forecast


In [ ]:
# Plot forecast with confidence intervals
fig, ax = plt.subplots(figsize=(14, 7))

# Historical data
ax.plot(df_daily['ds'], df_daily['y'], 'o-', label='Historical', alpha=0.7, markersize=2)

# Forecast
forecast_future = forecast.iloc[-90:]
ax.plot(forecast_future['ds'], forecast_future['yhat'], 'r-', label='Forecast', linewidth=2)

# Confidence intervals
ax.fill_between(forecast_future['ds'], 
                forecast_future['yhat_lower'], 
                forecast_future['yhat_upper'], 
                alpha=0.3, color='red', label='Confidence Interval')

ax.set_title('90-Day Sales Forecast', fontsize=16, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Total Sales (ETB)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Forecast summary
summary = forecaster.get_forecast_summary(forecast)

print('\n📊 FORECAST SUMMARY (Next 90 Days)')
print('='*50)
print(f'Total Forecasted Sales: ETB {summary["total_forecasted_sales"]:,.2f}')
print(f'Average Daily Sales: ETB {summary["avg_daily_sales"]:,.2f}')
print(f'Min Daily Sales: ETB {summary["min_daily_sales"]:,.2f}')
print(f'Max Daily Sales: ETB {summary["max_daily_sales"]:,.2f}')
print(f'Trend: {summary["trend"].upper()}')
print('='*50)


## 7. Save Forecast Results


In [ ]:
# Save forecast
import os
os.makedirs('../data/forecasts', exist_ok=True)

forecast.to_csv('../data/forecasts/forecast_results.csv', index=False)
print('✓ Forecast saved to ../data/forecasts/forecast_results.csv')


## Summary

### Results:
- ✓ Prophet model trained successfully
- ✓ 90-day forecast generated
- ✓ Model performance evaluated
- ✓ Predictions saved for use in dashboard and API

**Next Steps:** Proceed to notebook 04 for visualization and insights
